In [1]:
import pandas as pd

df = pd.read_excel("Gen_AI Dataset.xlsx")


In [4]:
df.to_csv("Gen_AI_Dataset.csv", index=False)

print("Excel converted to CSV successfully!")

Excel converted to CSV successfully!


In [5]:
df = pd.read_csv("Gen_AI_Dataset.csv")


In [6]:
print(df.head())
print(df.columns)
print(df.isnull().sum())


                                               Query  \
0  I am hiring for Java developers who can also c...   
1  I am hiring for Java developers who can also c...   
2  I am hiring for Java developers who can also c...   
3  I am hiring for Java developers who can also c...   
4  I am hiring for Java developers who can also c...   

                                      Assessment_url  
0  https://www.shl.com/solutions/products/product...  
1  https://www.shl.com/solutions/products/product...  
2  https://www.shl.com/solutions/products/product...  
3  https://www.shl.com/solutions/products/product...  
4  https://www.shl.com/products/product-catalog/v...  
Index(['Query', 'Assessment_url'], dtype='object')
Query             0
Assessment_url    0
dtype: int64


In [11]:
df.columns = df.columns.str.strip().str.lower()


In [13]:
df = df.dropna(subset=["query", "assessment_url"])


In [14]:
import re

def clean_text(text):
    if pd.isna(text):
        return ""
    text = re.sub(r"<.*?>", " ", str(text))  
    text = re.sub(r"\s+", " ", text)          
    return text.strip()


In [15]:
df["clean_query"] = df["query"].apply(clean_text)


In [17]:
def infer_test_type(text):
    text = str(text).lower()

    technical_keywords = [
        "coding", "programming", "python", "java", "sql", "data",
        "algorithm", "machine learning", "ai", "technical", "developer"
    ]

    behavioral_keywords = [
        "behavioral", "behavioural", "personality", "leadership",
        "communication", "teamwork", "attitude"
    ]

    for word in technical_keywords:
        if word in text:
            return "technical"

    for word in behavioral_keywords:
        if word in text:
            return "behavioral"

    return "general"


In [18]:
df["test_type"] = df["query"].apply(infer_test_type)


In [19]:
df[["query", "test_type"]].head()



,query,test_type
0,I am hiring for Java developers who can also c...,technical
1,I am hiring for Java developers who can also c...,technical
2,I am hiring for Java developers who can also c...,technical
3,I am hiring for Java developers who can also c...,technical
4,I am hiring for Java developers who can also c...,technical


In [20]:
df["final_text"] = (
    df["clean_query"].fillna("") + " " +
    df["test_type"].fillna("")
)


In [21]:
df.to_csv("clean_assessment_catalog.csv", index=False)


In [22]:
pip install sentence-transformers


  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/12.0 MB 7.7 MB/s eta 0:00:02
   --------- ------------------------------ 2.9/12.0 MB 7.0 MB/s eta 0:00:02
   ------------ --------------------------- 3.7/12.0 MB 6.2 MB/s eta 0:00:02
   -------------- ------------------------- 4.5/12.0 MB 5.2 MB/s eta 0:00:02
   ------------------- -------------------- 5.8/12.0 MB 5.4 MB/s eta 0:00:02
   ------------------------- -------------- 7.6/12.0 MB 5.9 MB/s eta 0:00:01
   ---------------------------- ----------- 8.7/12.0 MB 6.0 MB/s eta 0:00:01
   ------------------------------- -------- 9.4/12.0 MB 5.7 MB/s eta 0:00:01
   --------------------------------- ------ 10.0/12.0 MB 5.1 MB/s eta 0:00:01
   ---------------------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
pip install protobuf==5.28.3


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [27]:
pip install tf-keras


   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------------------------ --------- 1.3/1.7 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 4.2 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")


In [29]:
assessment_texts = df["final_text"].tolist()


In [30]:
assessment_embeddings = model.encode(
    assessment_texts,
    convert_to_tensor=True,   # makes similarity computation easy
    show_progress_bar=True
)


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [31]:
catalog = {
    "dataframe": df,
    "embeddings": assessment_embeddings
}


In [32]:
print(assessment_embeddings.shape)


torch.Size([65, 384])


In [33]:
model = SentenceTransformer("all-MiniLM-L6-v2")


In [34]:
user_query = "Hiring a Java developer with good communication skills"


In [35]:
query_embedding = model.encode(user_query)
print(query_embedding.shape) 

(384,)


In [36]:

from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity([query_embedding], assessment_embeddings)
print(similarity_scores.shape)  
print(similarity_scores)        


(1, 65)
[[0.60275674 0.60275674 0.60275674 0.60275674 0.60275674 0.43730587
  0.43730587 0.43730587 0.43730587 0.43730587 0.43730587 0.43730587
  0.43730587 0.43730587 0.18156143 0.18156143 0.18156143 0.18156143
  0.18156143 0.18156143 0.3400535  0.3400535  0.3400535  0.3400535
  0.3400535  0.3584814  0.3584814  0.35848147 0.3584814  0.3584814
  0.39008904 0.39008904 0.39008904 0.39008904 0.39008904 0.39008904
  0.39008904 0.39008904 0.39008904 0.22827457 0.22827457 0.22827457
  0.22827457 0.22827457 0.22827457 0.23481837 0.23481837 0.23481837
  0.23481837 0.23481837 0.34628698 0.34628698 0.34628698 0.34628698
  0.34628698 0.40236947 0.40236947 0.40236947 0.40236947 0.40236947
  0.40236947 0.40236947 0.40236947 0.40236947 0.40236947]]


In [37]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [38]:
# Compute similarity
similarity_scores = cosine_similarity([query_embedding], assessment_embeddings)

# flatten to 1D array
similarity_scores = similarity_scores.flatten()


In [40]:
print(df.columns)


Index(['query', 'assessment_url', 'clean_query', 'test_type', 'final_text'], dtype='object')


In [41]:
# Get indices of assessments sorted by similarity (highest first)
ranked_indices = np.argsort(similarity_scores)[::-1]

# For example, top 5 assessments:
top_5_indices = ranked_indices[:5]
print("Top 5 most relevant assessments:")
for idx in top_5_indices:
    print(f"{df.iloc[idx]['clean_query']} - Score: {similarity_scores[idx]:.3f}")




Top 5 most relevant assessments:
I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes. - Score: 0.603
I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes. - Score: 0.603
I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes. - Score: 0.603
I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes. - Score: 0.603
I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes. - Score: 0.603


In [1]:
TECHNICAL_KEYWORDS = [
    "java", "python", "sql", "coding", "developer", "programming",
    "backend", "frontend", "api", "data", "oop"
]

BEHAVIORAL_KEYWORDS = [
    "communication", "teamwork", "collaboration", "stakeholder",
    "leadership", "interpersonal", "behavioral", "soft skills"
]


In [2]:
def detect_query_intent(query):
    query = query.lower()

    technical = any(word in query for word in TECHNICAL_KEYWORDS)
    behavioral = any(word in query for word in BEHAVIORAL_KEYWORDS)

    return {
        "technical": technical,
        "behavioral": behavioral
    }


In [4]:
user_query = "Hiring a Java developer who can collaborate with business teams"

query_intent = detect_query_intent(user_query)
print(query_intent)


{'technical': True, 'behavioral': False}


In [9]:
import pandas as pd

df = pd.read_csv("clean_assessment_catalog.csv")


In [10]:
df.head()


,query,assessment_url,clean_query,test_type,final_text
0,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...,I am hiring for Java developers who can also c...,technical,I am hiring for Java developers who can also c...
1,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...,I am hiring for Java developers who can also c...,technical,I am hiring for Java developers who can also c...
2,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...,I am hiring for Java developers who can also c...,technical,I am hiring for Java developers who can also c...
3,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...,I am hiring for Java developers who can also c...,technical,I am hiring for Java developers who can also c...
4,I am hiring for Java developers who can also c...,https://www.shl.com/products/product-catalog/v...,I am hiring for Java developers who can also c...,technical,I am hiring for Java developers who can also c...


In [11]:
df['test_type'].value_counts()


test_type
technical    39
general      26
Name: count, dtype: int64

In [12]:
TECHNICAL_KEYWORDS = [
    "java", "python", "sql", "coding", "developer", "programming"
]

BEHAVIORAL_KEYWORDS = [
    "communication", "teamwork", "collaboration", "leadership"
]

def infer_test_type(text):
    text = text.lower()
    if any(k in text for k in TECHNICAL_KEYWORDS):
        return "Technical"
    elif any(k in text for k in BEHAVIORAL_KEYWORDS):
        return "Behavioral"
    else:
        return "Cognitive"

df['test_type'] = df['final_text'].apply(infer_test_type)


In [13]:
df['test_type']


0     Technical
1     Technical
2     Technical
3     Technical
4     Technical
        ...    
60    Technical
61    Technical
62    Technical
63    Technical
64    Technical
Name: test_type, Length: 65, dtype: object

In [34]:
def balanced_recommendation(
    df,
    min_technical=2,
    min_behavioral=1,
    top_k=5
):
    df = df.copy()

    # Sort by similarity
    df = df.sort_values("similarity", ascending=False)

    final = []
    used_urls = set()

    # --- Technical ---
    for _, row in df[df["test_type"].str.lower() == "technical"].iterrows():
        if row["assessment_url"] not in used_urls:
            final.append(row)
            used_urls.add(row["assessment_url"])
        if len([r for r in final if r["test_type"].lower() == "technical"]) == min_technical:
            break

    # --- Behavioral ---
    for _, row in df[df["test_type"].str.lower() == "behavioral"].iterrows():
        if row["assessment_url"] not in used_urls:
            final.append(row)
            used_urls.add(row["assessment_url"])
        if len([r for r in final if r["test_type"].lower() == "behavioral"]) == min_behavioral:
            break

    # --- Fill remaining ---
    for _, row in df.iterrows():
        if row["assessment_url"] not in used_urls:
            final.append(row)
            used_urls.add(row["assessment_url"])
        if len(final) == top_k:
            break

    return final


In [35]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
query_embedding = model.encode(user_query)


In [37]:
assessment_embeddings = model.encode(df['final_text'].tolist())


In [38]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_scores = cosine_similarity(
    [query_embedding],
    assessment_embeddings
).flatten()


In [40]:
df = df.copy()
df["similarity"] = similarity_scores

final_results = balanced_recommendation(
    df,
    min_technical=2,
    min_behavioral=1,
    top_k=5
)


In [41]:
print("\n🎯 FINAL RECOMMENDED ASSESSMENTS:\n")

for i, rec in enumerate(final_results, start=1):
    print(f"{i}. {rec['assessment_name']}")
    print(f"   Type: {rec['test_type']}")
    print(f"   URL: {rec['assessment_url']}")
    print(f"   Score: {rec['similarity']:.3f}\n")



🎯 FINAL RECOMMENDED ASSESSMENTS:

1. Core Java Entry Level New
   Type: Technical
   URL: https://www.shl.com/solutions/products/product-catalog/view/core-java-entry-level-new/
   Score: 0.700

2. Core Java Advanced Level New
   Type: Technical
   URL: https://www.shl.com/solutions/products/product-catalog/view/core-java-advanced-level-new/
   Score: 0.700

3. Occupational Personality Questionnaire Opq32R
   Type: Behavioral
   URL: https://www.shl.com/products/product-catalog/view/occupational-personality-questionnaire-opq32r/
   Score: 0.438

4. Java 8 New
   Type: Technical
   URL: https://www.shl.com/solutions/products/product-catalog/view/java-8-new/
   Score: 0.700

5. Automata Fix New
   Type: Technical
   URL: https://www.shl.com/solutions/products/product-catalog/view/automata-fix-new/
   Score: 0.700

